In [8]:
!pip install flask
!pip install flask-ngrok
!pip install mne

In [6]:
from google.colab import drive
drive.mount('/content/drive')

# Then load model from drive path like:
model_path = '/content/drive/MyDrive/final_model.h5'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pyngrok flask


In [10]:
!ngrok config check

Valid configuration file at /root/.config/ngrok/ngrok.yml


In [ ]:
from flask import Flask
from pyngrok import ngrok
import threading

app = Flask(__name__)

@app.route('/')
def home():
    return "Hello from Flask + ngrok!"

def run_app():
    app.run(port=5000)

# Start flask in a thread
threading.Thread(target=run_app).start()

# Open ngrok tunnel
public_url = ngrok.connect(5000).public_url
print("ngrok tunnel:", public_url)


 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


ngrok tunnel: https://985113bb09a6.ngrok-free.app


In [1]:
!fuser -k 5000/tcp


In [7]:
from tensorflow.keras.models import load_model
import os

model_path = "/content/drive/MyDrive/final_model.h5"

if os.path.exists(model_path):
    model = load_model(model_path)
    print(" Model loaded successfully")
else:
    model = None
    print(" Model not found at", model_path)


 Model loaded successfully


In [1]:
# Install required packages (Colab-friendly; harmless elsewhere)
!pip install flask pyngrok tensorflow mne scikit-learn flask-cors -q

import os
import time
import threading
import warnings
import tempfile
import numpy as np
import mne

from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok, conf
from tensorflow.keras.models import load_model
from absl.logging import set_verbosity, ERROR

# Setup
warnings.filterwarnings('ignore')
set_verbosity(ERROR)

app = Flask(__name__)
CORS(app)

#  CONFIG: update this path if needed (must exist)
MODEL_PATH = "/content/drive/MyDrive/final_model.h5"
TARGET_SFREQ = 100          # common training rate (adjust if yours differs)
BANDPASS = (0.3, 35.0)      # Hz
NOTCH = None                # set to 50 or 60 if your data needs it (e.g., 50 in EU)
EPOCH_LEN_SEC = 30          # standard hypnogram epoch length

model = None
model_timesteps = None  # inferred from model.input_shape

# Load ONLY your model (no fallbacks)
try:
    if os.path.exists(MODEL_PATH):
        model = load_model(MODEL_PATH, compile=False)
        print("Model loaded:", MODEL_PATH)

        # Infer timesteps from model input shape: (None, T, 1)
        ish = model.input_shape
        # ish can be a list in some multi-input models
        if isinstance(ish, list):
            ish = ish[0]
        # ish example: (None, timesteps, channels)
        if len(ish) >= 3:
            model_timesteps = ish[1]
            print("ℹ Model expects timesteps:", model_timesteps, "channels:", ish[-1])
        else:
            print("Could not infer model timesteps from input_shape:", ish)
            model_timesteps = None
    else:
        print(f"Model not found at: {MODEL_PATH}")
        model = None
except Exception as e:
    print(f"Error loading model: {e}")
    model = None


# --------- Sleep stage mapping used to select annotated epochs ----------
stage_dict = {
    'Sleep stage W': 0,
    'Sleep stage 1': 1,
    'Sleep stage 2': 2,
    'Sleep stage 3': 3,
    'Sleep stage 4': 4,
    'Sleep stage R': 5
}


def _preprocess_epoch(epoch_1d, sfreq, target_sfreq):
    """
    Per-epoch preprocessing:
    - Bandpass 0.3–35 Hz (adjustable)
    - Optional notch at 50/60 Hz
    - Resample to target_sfreq
    - Per-epoch z-score: (x - mean) / std
    Returns epoch with shape (target_len,)
    """
    sig = epoch_1d.astype(np.float32)

    # Use an MNE RawArray to leverage high-quality filters
    info = mne.create_info(ch_names=['EEG'], sfreq=sfreq, ch_types=['eeg'])
    raw = mne.io.RawArray(sig[np.newaxis, :], info, verbose=False)

    # Band-pass
    raw.filter(BANDPASS[0], BANDPASS[1], fir_design='firwin', verbose=False)

    # Notch (optional)
    if NOTCH in (50, 60):
        raw.notch_filter(freqs=[NOTCH], verbose=False)

    # Resample if needed
    if target_sfreq is not None and abs(raw.info['sfreq'] - target_sfreq) > 1e-6:
        raw.resample(target_sfreq, npad="auto", verbose=False)

    x = raw.get_data()[0]
    # Per-epoch z-score
    m = np.mean(x)
    s = np.std(x)
    if s < 1e-7:
        s = 1e-7
    x = (x - m) / s
    return x


def _pad_or_crop(x, target_len):
    """Pad (reflect) or crop 1D array to target length."""
    n = x.shape[0]
    if n == target_len:
        return x
    if n > target_len:
        return x[:target_len]
    # pad
    pad = target_len - n
    # reflect padding to avoid edges
    left = pad // 2
    right = pad - left
    return np.pad(x, (left, right), mode='reflect')


def extract_epochs_from_subject(psg_path, ann_path, channel_pick='EEG Fpz-Cz'):
    """
    Returns ndarray of shape (n_epochs, samples) BEFORE channel-last expansion.
    """
    try:
        raw = mne.io.read_raw_edf(psg_path, preload=True, verbose=False)
        annotations = mne.read_annotations(ann_path)
        if annotations is None or len(annotations) == 0:
            print(" No annotations in hypnogram.")
            return np.array([])

        raw.set_annotations(annotations)

        # Channel selection with fallback
        if channel_pick in raw.ch_names:
            raw.pick_channels([channel_pick])
        else:
            raw.pick_types(eeg=True)
            if len(raw.ch_names) == 0:
                print(" No EEG channels found.")
                return np.array([])

        eeg = raw.get_data()[0]
        sfreq = float(raw.info['sfreq'])
        epoch_samples = int(round(EPOCH_LEN_SEC * sfreq))

        epochs = []
        # Loop through annotations and slice 30s epochs for known stages
        for annot in annotations:
            desc = annot['description']
            if desc not in stage_dict:
                continue
            start = int(round(annot['onset'] * sfreq))
            dur = int(round(annot['duration'] * sfreq))
            # iterate in 30s chunks
            for i in range(0, dur, epoch_samples):
                a = start + i
                b = a + epoch_samples
                if b <= eeg.shape[0]:
                    seg = eeg[a:b]
                    if seg.shape[0] == epoch_samples:
                        epochs.append(seg)

        if not epochs:
            return np.array([])

        return np.stack(epochs, axis=0)  # (n_epochs, samples)
    except Exception as e:
        print(f" Error processing EDF: {e}")
        return np.array([])


def build_model_inputs(epochs_raw, raw_sfreq):
    """
    - Preprocess each epoch
    - Resample to TARGET_SFREQ
    - Conform to model_timesteps (pad/crop)
    - Expand channel dim -> (n, T, 1)
    """
    if model_timesteps is None:
        # If model doesn't specify, infer from first preprocessed epoch length
        target_len = int(round(EPOCH_LEN_SEC * (TARGET_SFREQ or raw_sfreq)))
    else:
        target_len = model_timesteps

    processed = []
    for ep in epochs_raw:
        x = _preprocess_epoch(ep, sfreq=raw_sfreq, target_sfreq=TARGET_SFREQ or raw_sfreq)
        x = _pad_or_crop(x, target_len)
        processed.append(x)

    X = np.stack(processed, axis=0).astype(np.float32)  # (n, T)
    X = X[..., np.newaxis]  # (n, T, 1)
    return X, target_len


@app.route('/')
def home():
    return "Sleep Disorder Detection API is running."


@app.route('/predict', methods=['POST'])
def predict():
    if model is None:
        return jsonify({'error': 'Model not loaded'}), 500

    if 'psgFile' not in request.files or 'hypnoFile' not in request.files:
        return jsonify({'error': 'Missing files'}), 400

    try:
        with tempfile.TemporaryDirectory() as tmpdir:
            psg_path = os.path.join(tmpdir, 'psg.edf')
            hypno_path = os.path.join(tmpdir, 'hypnogram.edf')
            request.files['psgFile'].save(psg_path)
            request.files['hypnoFile'].save(hypno_path)

            # Extract and preprocess
            epochs_raw = extract_epochs_from_subject(psg_path, hypno_path)
            if epochs_raw.size == 0:
                return jsonify({'error': 'No valid epochs'}), 400

            raw_tmp = mne.io.read_raw_edf(psg_path, preload=False, verbose=False)
            raw_sfreq = float(raw_tmp.info['sfreq'])

            X, T = build_model_inputs(epochs_raw, raw_sfreq)  # Ensure this matches training preprocessing exactly

            # Predict probabilities
            probs = model.predict(X, verbose=0).flatten()
            epoch_labels = (probs >= 0.5).astype(int)  # threshold at 0.5
            is_disordered = bool(np.mean(epoch_labels) > 0.5)  # majority voting

            message = "Likely abnormal sleep pattern." if is_disordered else "Likely normal sleep pattern."

            return jsonify({
                'disorder_probability_mean': float(np.mean(probs)),
                'is_disordered': is_disordered,
                'epochs_processed': int(X.shape[0]),
                'message': message
            })

    except Exception as e:
        return jsonify({'error': 'Processing failed', 'details': str(e)}), 500



@app.route('/interface')
def web_interface():
    # (your original HTML kept intact; only added Confidence + extra line)
    return """
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8" />
<meta name="viewport" content="width=device-width, initial-scale=1" />
<title>Sleep Disorder Detection</title>
<style>
  * { box-sizing: border-box; }
  body {
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    background: #f9faff; color: #333; margin: 0; padding: 20px;
    display: flex; justify-content: center;
  }
  .container {
    background: #fff; max-width: 500px; width: 100%;
    border-radius: 10px; box-shadow: 0 8px 20px rgb(0 0 0 / 0.1);
    padding: 30px 40px;
  }
  h1 { text-align: center; color: #2c3e50; margin-bottom: 10px; }
  p.description { text-align: center; color: #6c7a89; margin-bottom: 30px; font-size: 1.1rem; }
  form.upload-form { display: flex; flex-direction: column; gap: 20px; }
  label { display: block; margin-bottom: 8px; font-weight: 600; color: #34495e; }
  input[type="file"] {
    width: 100%; padding: 10px 12px; border: 2px solid #ccc; border-radius: 6px;
    transition: border-color 0.3s ease; cursor: pointer;
  }
  input[type="file"]:focus, input[type="file"]:hover { border-color: #3498db; outline: none; }
  button {
    background: #3498db; border: none; padding: 12px 0; color: white; font-weight: 700;
    border-radius: 8px; cursor: pointer; font-size: 1.1rem; transition: background-color 0.3s ease;
  }
  button:hover { background: #2980b9; }
  .result { margin-top: 30px; padding: 20px 25px; border-radius: 8px; font-weight: 600; font-size: 1.15rem; display: none; }
  .result.disordered { background-color: #ffe6e6; border-left: 6px solid #e74c3c; color: #c0392b; }
  .result.normal { background-color: #e8f6e8; border-left: 6px solid #27ae60; color: #229954; }
  .result h3 { margin-top: 0; margin-bottom: 10px; font-weight: 700; }
  .loading { display: none; text-align: center; margin: 20px 0; }
  .spinner {
    border: 4px solid rgba(0, 0, 0, 0.1); width: 36px; height: 36px; border-radius: 50%;
    border-left-color: #3498db; animation: spin 1s linear infinite; margin: 0 auto;
  }
  @keyframes spin { 0% { transform: rotate(0deg);} 100% { transform: rotate(360deg);} }
  @media (max-width: 480px) { .container { padding: 20px; } }
</style>
</head>
<body>
  <div class="container">
    <h1>Sleep Disorder Detection</h1>
    <p class="description">Upload PSG and hypnogram files (EDF format) for analysis</p>

    <form id="uploadForm" class="upload-form">
      <div>
        <label for="psgFile">PSG File (EDF):</label>
        <input type="file" id="psgFile" name="psgFile" accept=".edf" required />
      </div>
      <div>
        <label for="hypnoFile">Hypnogram File (EDF):</label>
        <input type="file" id="hypnoFile" name="hypnoFile" accept=".edf" required />
      </div>
      <button type="submit" id="analyzeBtn">Analyze</button>
    </form>

    <div class="loading" id="loadingIndicator">
      <div class="spinner"></div>
      <p>Processing your files...</p>
    </div>

    <div id="resultContainer" class="result">
      <h3>Analysis Results</h3>
      <p>Disorder Probability: <span id="probability">0</span></p>
      <p>Confidence: <span id="confidence">0</span></p>
      <p>Epochs Processed: <span id="epochCount">0</span></p>
      <p>Diagnosis: <span id="diagnosis">-</span></p>
      <p id="details"></p>
    </div>

    <div id="errorContainer" class="result" style="display: none; background-color: #fff3cd; border-left: 6px solid #ffc107; color: #856404;">
      <h3>Error</h3>
      <p id="errorMessage"></p>
      <p id="errorDetails"></p>
    </div>
  </div>

  <script>
    document.getElementById('uploadForm').addEventListener('submit', async function(e) {
      e.preventDefault();

      document.getElementById('loadingIndicator').style.display = 'block';
      document.getElementById('resultContainer').style.display = 'none';
      document.getElementById('errorContainer').style.display = 'none';

      const btn = document.getElementById('analyzeBtn');
      btn.disabled = true; btn.textContent = 'Processing...';

      try {
        const fd = new FormData();
        fd.append('psgFile', document.getElementById('psgFile').files[0]);
        fd.append('hypnoFile', document.getElementById('hypnoFile').files[0]);

        const res = await fetch('/predict', { method: 'POST', body: fd });
        const data = await res.json();

        if (res.ok) {
          const resultDiv = document.getElementById('resultContainer');
          document.getElementById('probability').textContent = (data.disorder_probability * 100).toFixed(2) + '%';
          document.getElementById('confidence').textContent = (data.confidence * 100).toFixed(2) + '%';
          document.getElementById('epochCount').textContent = data.epochs_processed;

          if (data.is_disordered) {
            document.getElementById('diagnosis').textContent = 'Sleep disorder detected';
            resultDiv.className = 'result disordered';
          } else {
            document.getElementById('diagnosis').textContent = 'Normal sleep pattern';
            resultDiv.className = 'result normal';
          }

          document.getElementById('details').textContent = data.message || '';
          resultDiv.style.display = 'block';
          document.getElementById('errorContainer').style.display = 'none';
        } else {
          const errDiv = document.getElementById('errorContainer');
          document.getElementById('errorMessage').textContent = data.error || 'An error occurred';
          document.getElementById('errorDetails').textContent = data.details || '';
          errDiv.style.display = 'block';
          document.getElementById('resultContainer').style.display = 'none';
        }
      } catch (err) {
        const errDiv = document.getElementById('errorContainer');
        document.getElementById('errorMessage').textContent = 'Network error';
        document.getElementById('errorDetails').textContent = err.message;
        errDiv.style.display = 'block';
        document.getElementById('resultContainer').style.display = 'none';
      } finally {
        document.getElementById('loadingIndicator').style.display = 'none';
        btn.disabled = false; btn.textContent = 'Analyze';
        const visible = document.getElementById('resultContainer').style.display === 'block'
          ? 'resultContainer' : 'errorContainer';
        document.getElementById(visible).scrollIntoView({ behavior: 'smooth' });
      }
    });
  </script>
</body>
</html>
    """


def run_flask():
    app.run(host='0.0.0.0', port=5000, debug=False, threaded=True)


if __name__ == '__main__':
    # Kill any previous server on 5000 (works on Colab)
    os.system('fuser -k 5000/tcp 2>/dev/null || true')

    # Ngrok region (adjust as needed)
    conf.get_default().region = "us"

    # Start Flask in a background thread
    flask_thread = threading.Thread(target=run_flask, daemon=True)
    flask_thread.start()

    # Wait for the server
    time.sleep(3)

    try:
        public_url = ngrok.connect(5000, bind_tls=True).public_url
        print(f"\n Web Interface URL: {public_url}/interface")
        print(f" API Endpoint: {public_url}/predict")
        print("\nKeep this Colab tab open to maintain the connection!")
    except Exception as e:
        print(f" Ngrok error: {e}")
        print("Try accessing via local URL: http://127.0.0.1:5000/interface")


Model loaded: /content/drive/MyDrive/final_model.h5
ℹ Model expects timesteps: 750 channels: 1
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
ERROR:pyngrok.process.ngrok:t=2025-08-14T16:47:02+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/agents\r\n\r\nERR_NGROK_108\r\n"
ERROR:pyngrok.process.ngrok:t=2025-08-14T16:47:02+0000 lvl=eror msg="session closing" obj=

 Ngrok error: The ngrok process errored on start: authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/agents\r\n\r\nERR_NGROK_108\r\n.
Try accessing via local URL: http://127.0.0.1:5000/interface


In [4]:
import requests
print(requests.get("https://d8b71696a6f6.ngrok-free.app").text)
# Should return "Sleep Disorder Detection API is running."

<!DOCTYPE html>
<html class="h-full" lang="en-US" dir="ltr">
  <head>
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Regular-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-RegularItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Medium-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Semibold-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-MediumItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/ibm-plex-mono/IBMPlexMono-Tex

In [2]:
# 1️ Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 2️ Ask user for ngrok token (only if not already saved)
import os

token_file = "/content/drive/MyDrive/ngrok_token.txt"
if not os.path.exists(token_file):
    token = input("Enter your ngrok token: ").strip()
    with open(token_file, "w") as f:
        f.write(token)
    print(f" Token saved to {token_file}")
else:
    with open(token_file) as f:
        token = f.read().strip()
    print(f" Token loaded from {token_file}")

# 3️ Just set auth token for later use
from pyngrok import ngrok
ngrok.set_auth_token(token)

# 4️ Do NOT start a new tunnel here, only print instructions
print(" Ngrok token set. The tunnel will be started in the final cell.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 Token loaded from /content/drive/MyDrive/ngrok_token.txt
 Ngrok token set. The tunnel will be started in the final cell.


In [4]:
from pyngrok import ngrok

# Start tunnel only here
public_url = ngrok.connect(5000).public_url
print(" ngrok tunnel URL:", public_url + "/interface")


 ngrok tunnel URL: https://ab302e345492.ngrok-free.app/interface


In [5]:
import requests

url = "https://d734e82445c1.ngrok-free.app"
try:
    r = requests.get(url)
    print("Status:", r.status_code)
    print("Response:", r.text)
except Exception as e:
    print("Error:", e)


Status: 404
Response: <!DOCTYPE html>
<html class="h-full" lang="en-US" dir="ltr">
  <head>
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Regular-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-RegularItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Medium-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Semibold-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-MediumItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/ibm-ple